<a href="https://colab.research.google.com/github/sagar9926/END_Session10/blob/main/END_Session_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
%%bash
python -m spacy download en
python -m spacy download de

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')
✔ Download and installation successful
You can now load the model via spacy.load('de_core_news_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/de_core_news_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/de
You can now load the model via spacy.load('de')


In [28]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torchtext.datasets import Multi30k
from torchtext.data import Field , BucketIterator

import spacy
import numpy as np

import random
import math
import time

In [29]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [30]:
spacy_de = spacy.load('de')
spacy_en = spacy.load('en')

In [31]:
def tokenize_de(text):
    """
    Tokenizes German text from a string into a list of strings (tokens) and reverses it
    """
    return [tok.text for tok in spacy_de.tokenizer(text)][::-1]

def tokenize_en(text):
    """
    Tokenizes English text from a string into a list of strings (tokens)
    """
    return [tok.text for tok in spacy_en.tokenizer(text)]

In [32]:
TRG = Field(tokenize = tokenize_de, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

SRC = Field(tokenize = tokenize_en, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

In [33]:
train_data, valid_data, test_data = Multi30k.splits(exts = ('.en','.de'), 
                                                    fields = (SRC, TRG))

In [34]:
print(f"Number of training examples: {len(train_data.examples)}")
print(f"Number of validation examples: {len(valid_data.examples)}")
print(f"Number of testing examples: {len(test_data.examples)}")

Number of training examples: 29000
Number of validation examples: 1014
Number of testing examples: 1000


In [35]:
print(vars(train_data.examples[0]))

{'src': ['two', 'young', ',', 'white', 'males', 'are', 'outside', 'near', 'many', 'bushes', '.'], 'trg': ['.', 'büsche', 'vieler', 'nähe', 'der', 'in', 'freien', 'im', 'sind', 'männer', 'weiße', 'junge', 'zwei']}


In [36]:
SRC.build_vocab(train_data, min_freq = 2)
TRG.build_vocab(train_data, min_freq = 2)

In [37]:
print(f"Unique tokens in source (de) vocabulary: {len(SRC.vocab)}")
print(f"Unique tokens in target (en) vocabulary: {len(TRG.vocab)}")

Unique tokens in source (de) vocabulary: 5893
Unique tokens in target (en) vocabulary: 7855


In [38]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [39]:
BATCH_SIZE = 128

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE, 
    device = device)

In [40]:
class Encoder(nn.Module):
  def __init__(self , input_dim,emb_dim , enc_hid_dim ,dec_hid_dim, dropout):
    super().__init__()
    self.enc_hid_dim = enc_hid_dim
    self.dec_hid_dim = dec_hid_dim
    self.emb_dim = emb_dim
    self.embedding = nn.Embedding(input_dim,emb_dim)
    self.rnn = nn.GRU(emb_dim ,enc_hid_dim , bidirectional = True)
    self.fc = nn.Linear(enc_hid_dim*2 , dec_hid_dim)
    self.dropout = nn.Dropout(dropout)

  def forward(self, src):

    embedding = self.dropout(self.embedding(src))
    # Input Dimension : SRC --> [sequence_length , batch_size]
    # Output Dimension : embedding --> [sequence_length , batch_size , embedding dimension]

    outputs , hidden = self.rnn(embedding)
    # Outputs Dimension : outputs --> [sequence_length , batch_size , enc_hid_dim * n directions ]
    # Hidden Dimension :  hidden --> [  n layers * n directions ,batch_size , enc_hid_dim ]

    hidden  = torch.tanh(self.fc(torch.cat((hidden[-2 ,:,:] , hidden[-1 ,:,:]), dim = 1)))
    # hidden[-2 :, : ] --> [batch_size , enc_hid_dim ]
    # hidden[-1 :, : ] --> [batch_size , enc_hid_dim ]
    # Input to Fully connected layer --> torch.cat((hidden[-2 :, : ] , hidden[-1 :, : ]), dim = 1) --> [batch_size , enc_hid_dim * 2]
    # hidden  --> [batch_size , dec_hid_dim ]

    return (outputs , hidden)


In [41]:
class Attention(nn.Module):

  def __init__(self,enc_hid_dim,dec_hid_dim):
    super().__init__()
    self.attn = nn.Linear(enc_hid_dim * 2 + dec_hid_dim , dec_hid_dim)
    self.v = nn.Linear(dec_hid_dim , 1 , bias = False)

  def forward(self, hidden , encoder_outputs): # hidden is Decoder's hidden 
    src_len = encoder_outputs.shape[0]

    hidden = hidden.unsqueeze(1).repeat(1,src_len,1) # hidden is Decoder's hidden basically S_t-1
    # Input --> [batch_size , dec_hid_dim]
    # After unsqueeze --> [batch_size , 1 ,  dec_hid_dim]
    # After repeat --> [batch_size , sequence_length ,  dec_hid_dim]
    
    encoder_outputs = encoder_outputs.permute(1,0,2)
    # encoder_outputs --> [sequence_length , batch_size , enc_hid_dim * n directions ]
    # encoder_outputs --> (after permute) : [batch_size , sequence_length , enc_hid_dim * n directions ]

    energy = torch.tanh(self.attn(torch.cat((hidden , encoder_outputs ) , dim = 2))) # Basically the similarity of decoder hidden vector(repeated) with encoder hidden vectors
    # energy --> [batch_size , sequence_length ,  dec_hid_dim]

    attention = self.v(energy).squeeze(2)
    # attention --> [batch_size , sequence_length ,  1]
    # attention --> (after squeeze)  [batch_size , sequence_length ] 
    # we want our attention vector to be as long as our input sequence

    return(F.softmax(attention , dim = 1)) 



In [42]:
class Decoder(nn.Module):
  def __init__(self , output_dim,emb_dim , enc_hid_dim ,dec_hid_dim , dropout , attention):
    super().__init__()
    self.output_dim = output_dim
    self.attention = attention
    self.embedding = nn.Embedding(output_dim,emb_dim)
    self.rnn = nn.GRU((enc_hid_dim * 2) + emb_dim , dec_hid_dim)
    self.fc_out = nn.Linear((enc_hid_dim * 2) + emb_dim  + dec_hid_dim , output_dim)
    self.dropout = nn.Dropout(dropout)

  def forward(self , input , hidden , encoder_outputs):
    #input --> [batch size]
    input = input.unsqueeze(0)
    #input --> [1 , batch size]

    embedded = self.dropout(self.embedding(input))
    # embedding --> [1 , batch size , emb_dim ]

    a = self.attention(hidden , encoder_outputs) # hidden is Decoder's hidden 
    # a -->  [batch_size , sequence_length ] 

    a = a.unsqueeze(1)
    # a -->  [ batch_size , 1 , sequence_length ]

    # encoder_outputs --> [sequence_length , batch_size , enc_hid_dim * 2 ] # ndirections = 2
    encoder_outputs = encoder_outputs.permute(1,0,2)
    # encoder_outputs --> (after permute) : [batch_size , sequence_length , enc_hid_dim * 2 ]

    weighted = torch.bmm(a , encoder_outputs)
    # weighted --> [ batch_size , 1 , enc_hid_dim * 2]

    weighted = weighted.permute(1,0,2)
    # weighted --> [ 1 , batch_size , enc_hid_dim * 2]

    rnn_input = torch.cat((embedded , weighted), dim = 2)
    # rnn_input --> [ 1 , batch_size , enc_hid_dim * 2 + emb_dim ]

    output , hidden = self.rnn(rnn_input , hidden.unsqueeze(0)) # hidden  --> [batch_size , dec_hid_dim ]

    embedded = embedded.squeeze(0)
    output = output.squeeze(0)
    weighted = weighted.squeeze(0)

    prediction = self.fc_out(torch.cat((output , weighted , embedded), dim = 1))

    return prediction , hidden.squeeze(0)



In [43]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use teacher forcing 75% of the time
        
        batch_size = src.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #encoder_outputs is all hidden states of the input sequence, back and forwards
        #hidden is the final forward and backward hidden states, passed through a linear layer
        encoder_outputs, hidden = self.encoder(src)
                
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden state and all encoder hidden states
            #receive output tensor (predictions) and new hidden state
            output, hidden = self.decoder(input, hidden, encoder_outputs)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1

        return outputs

In [44]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
ENC_HID_DIM = 512
DEC_HID_DIM = 512
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

attn = Attention(ENC_HID_DIM , DEC_HID_DIM) 
enc = Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM , ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, DEC_DROPOUT,attn)

model = Seq2Seq(enc, dec, device).to(device)

In [45]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)
        
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(5893, 256)
    (rnn): GRU(256, 512, bidirectional=True)
    (fc): Linear(in_features=1024, out_features=512, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (attention): Attention(
      (attn): Linear(in_features=1536, out_features=512, bias=True)
      (v): Linear(in_features=512, out_features=1, bias=False)
    )
    (embedding): Embedding(7855, 256)
    (rnn): GRU(1280, 512)
    (fc_out): Linear(in_features=1792, out_features=7855, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [46]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 24,036,783 trainable parameters


In [47]:
optimizer = optim.Adam(model.parameters())

In [48]:
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

In [49]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch.src
        trg = batch.trg
        
        optimizer.zero_grad()
        
        output = model(src, trg)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [50]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.src
            trg = batch.trg

            output = model(src, trg, 0) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)
            
            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [51]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [53]:
N_EPOCHS = 20
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 1m 37s
	Train Loss: 3.169 | Train PPL:  23.779
	 Val. Loss: 3.599 |  Val. PPL:  36.559
Epoch: 02 | Time: 1m 37s
	Train Loss: 2.625 | Train PPL:  13.802
	 Val. Loss: 3.383 |  Val. PPL:  29.447
Epoch: 03 | Time: 1m 37s
	Train Loss: 2.246 | Train PPL:   9.448
	 Val. Loss: 3.342 |  Val. PPL:  28.263
Epoch: 04 | Time: 1m 38s
	Train Loss: 1.944 | Train PPL:   6.986
	 Val. Loss: 3.350 |  Val. PPL:  28.514
Epoch: 05 | Time: 1m 37s
	Train Loss: 1.712 | Train PPL:   5.540
	 Val. Loss: 3.409 |  Val. PPL:  30.231
Epoch: 06 | Time: 1m 38s
	Train Loss: 1.550 | Train PPL:   4.711
	 Val. Loss: 3.390 |  Val. PPL:  29.664
Epoch: 07 | Time: 1m 38s
	Train Loss: 1.386 | Train PPL:   4.000
	 Val. Loss: 3.473 |  Val. PPL:  32.239
Epoch: 08 | Time: 1m 38s
	Train Loss: 1.246 | Train PPL:   3.478
	 Val. Loss: 3.504 |  Val. PPL:  33.232
Epoch: 09 | Time: 1m 36s
	Train Loss: 1.164 | Train PPL:   3.202
	 Val. Loss: 3.541 |  Val. PPL:  34.494
Epoch: 10 | Time: 1m 38s
	Train Loss: 1.076 | Train PPL